In [4]:
import random
import operator

# Given sequence data points: (N, output)
data_points = [
    (1, 3),
    (2, 7),
    (3, 13),
    (4, 21),
    (5, 31),
    (6, 43),
    (7, 59),
    (8, 73),
    (9, 91),
    (10, 111),
]

# Allowed functions (basic operations)
FUNCTIONS = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
}

# Terminals: variable 'n' and some constants (restricted to smaller range)
TERMINALS = ['n'] + [random.uniform(-2, 2) for _ in range(5)]  # Smaller constant range

POP_SIZE = 100
GENERATIONS = 20
PC = 0.7
PM = 0.3
MAX_DEPTH = 2  # Allow depth up to 2 for more complex terms

# Function to generate random chromosomes (expression trees)
def generate_random_chromosome(depth=0):
    if depth >= MAX_DEPTH:
        return random.choice(TERMINALS)
    else:
        func = random.choice(list(FUNCTIONS.keys()))
        return [func, generate_random_chromosome(depth + 1), generate_random_chromosome(depth + 1)]

# Evaluate expression for a given value of n
def express(chromosome, n):
    if isinstance(chromosome, float) or isinstance(chromosome, int):
        return chromosome
    elif chromosome == 'n':
        return n
    else:
        func = chromosome[0]
        val1 = express(chromosome[1], n)
        val2 = express(chromosome[2], n)
        return FUNCTIONS[func](val1, val2)

# Fitness function: Sum of squared errors with penalty for tree size
def tree_size(expr):
    return 1  # Since we're only allowing depth 2 expressions

def fitness_function(chromosome):
    error = 0.0
    for (n, val) in data_points:
        pred = express(chromosome, n)
        error += (val - pred) ** 2
    size = tree_size(chromosome)
    penalty = 0.1 * size  # Increase penalty for large expressions
    return -(error + penalty)  # Negative to maximize fitness

# Selection: Tournament selection
def select(population):
    tournament_size = 3
    selected = random.sample(population, tournament_size)
    selected.sort(key=lambda c: c['fitness'], reverse=True)
    return selected[0]['chromosome']

# Crossover: Subtree exchange between two parents
def crossover(parent1, parent2):
    return parent2, parent1  # Simple swap to keep it very basic

# Mutation: Replace a random subtree with a new random expression
def mutate(chromosome):
    return generate_random_chromosome()  # Replace entire tree with a new one

# Best chromosome (highest fitness)
def best_chromosome(pop):
    return max(pop, key=lambda c: c['fitness'])

# Main GP loop
def gene_expression_algorithm(n, G, Pc, Pm):
    population = []
    for _ in range(n):
        chromo = generate_random_chromosome()
        fitness = fitness_function(chromo)
        population.append({'chromosome': chromo, 'fitness': fitness})

    for gen in range(G):
        new_population = []

        while len(new_population) < n:
            parent1 = select(population)
            parent2 = select(population)

            if random.random() < Pc:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1, parent2

            if random.random() < Pm:
                child1 = mutate(child1)
            if random.random() < Pm:
                child2 = mutate(child2)

            fitness1 = fitness_function(child1)
            fitness2 = fitness_function(child2)

            new_population.append({'chromosome': child1, 'fitness': fitness1})
            if len(new_population) < n:
                new_population.append({'chromosome': child2, 'fitness': fitness2})

        population = new_population
        best = best_chromosome(population)
        print(f"Generation {gen + 1}, Best fitness: {-best['fitness']:.4f}")

    return best['chromosome']

# Pretty print the expression tree as a string
def expr_to_string(expr):
    if isinstance(expr, list):
        func = expr[0]
        return f"({expr_to_string(expr[1])} {func} {expr_to_string(expr[2])})"
    else:
        return str(expr)

# Run the symbolic regression
best_expr = gene_expression_algorithm(POP_SIZE, GENERATIONS, PC, PM)
print("\nBest expression found:")
print(expr_to_string(best_expr))


Generation 1, Best fitness: 3404.4721
Generation 2, Best fitness: 1156.0480
Generation 3, Best fitness: 529.3040
Generation 4, Best fitness: 529.3040
Generation 5, Best fitness: 529.3040
Generation 6, Best fitness: 529.3040
Generation 7, Best fitness: 529.3040
Generation 8, Best fitness: 529.3040
Generation 9, Best fitness: 529.3040
Generation 10, Best fitness: 509.5823
Generation 11, Best fitness: 43.2547
Generation 12, Best fitness: 43.2547
Generation 13, Best fitness: 43.2547
Generation 14, Best fitness: 509.5823
Generation 15, Best fitness: 509.5823
Generation 16, Best fitness: 509.5823
Generation 17, Best fitness: 509.5823
Generation 18, Best fitness: 509.5823
Generation 19, Best fitness: 509.5823
Generation 20, Best fitness: 509.5823

Best expression found:
((n * n) - (-0.11974350193956163 + -0.11974350193956163))
